<a href="https://colab.research.google.com/github/YoannLoadwig/Estagio-target-sistemas/blob/main/Code_NOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving status_nfe.csv to status_nfe.csv


In [4]:
!pip install requests beautifulsoup4 schedule

In [6]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import schedule
import time
from datetime import datetime

# URL da página da Receita Federal
URL = "https://www.nfe.fazenda.gov.br/portal/disponibilidade.aspx?versao=0.00&tipoConteudo=P2c98tUpxrI="

# Mapeamento dos status baseado no nome do arquivo da imagem
STATUS_MAP = {
    "bola_verde_p.png": 2,   # Operando normalmente
    "bola_amarela_p.png": 1, # Com falhas
    "bola_vermelho_p.png": 0 # Indisponível
}

CSV_FILE = "status_nfe.csv"
ALARM_FILE = "alarmes_nfe.csv"

def obter_status_nfe():
    """Obtém os status dos serviços da Receita Federal e retorna os dados formatados."""
    session = requests.Session()
    response = session.get(URL)

    if response.status_code != 200:
        print(f"Erro ao acessar a página: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")

    # Buscar a tabela pela classe
    tabela = soup.find("table", {"class": "tabelaListagemDados"})

    if not tabela:
        print("Tabela não encontrada!")
        return []

    dados = []

    # Iterar sobre as linhas da tabela
    for linha in tabela.find_all("tr")[1:]:  # Pula o cabeçalho
        colunas = linha.find_all("td")

        if len(colunas) < 3:
            continue  # Ignora linhas incompletas

        autorizador = colunas[0].text.strip()
        autorizacao4 = extrair_status(colunas[1])
        status_servico4 = extrair_status(colunas[2])
        data_hora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Data e hora da coleta

        # Verifica se precisa disparar um alarme
        if autorizacao4 in [0, 1] or status_servico4 in [0, 1]:
            ativar_alarme(autorizador, autorizacao4, status_servico4, data_hora)

        dados.append([autorizador, autorizacao4, status_servico4, data_hora])

    return dados

def extrair_status(coluna):
    """Extrai o nome do arquivo da imagem e converte para um valor numérico."""
    img = coluna.find("img")

    if img and "src" in img.attrs:
        src = img["src"].strip().lower()  # Obtém o caminho da imagem
        nome_arquivo = src.split("/")[-1]  # Extrai apenas o nome do arquivo

        return STATUS_MAP.get(nome_arquivo, -1)  # Retorna o número correspondente
    return -1  # Caso não encontre a imagem

def ativar_alarme(autorizador, autorizacao4, status_servico4, data_hora):
    """Exibe um alerta no terminal e registra no arquivo de alarmes."""
    print(f"🚨 ALARME: Problema detectado em {autorizador} - Autorização4: {autorizacao4}, Status Serviço4: {status_servico4} - {data_hora}")

    # Salvar no arquivo CSV de alarmes
    file_exists = os.path.exists(ALARM_FILE)
    with open(ALARM_FILE, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)

        if not file_exists:
            writer.writerow(["Autorizador", "Autorização4", "Status Serviço4", "Data/Hora"])

        writer.writerow([autorizador, autorizacao4, status_servico4, data_hora])

def salvar_csv(dados, arquivo=CSV_FILE):
    """Salva os dados no arquivo CSV sem sobrescrever, evitando duplicatas."""
    file_exists = os.path.exists(arquivo)

    with open(arquivo, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)

        # Se o arquivo não existe, escrever cabeçalho
        if not file_exists:
            writer.writerow(["Autorizador", "Autorização4", "Status Serviço4", "Data/Hora"])

        # Escrever os dados
        for linha in dados:
            writer.writerow(linha)

    print(f"Dados salvos em {arquivo}")

def executar_coleta():
    """Executa a coleta de dados, salva no CSV e registra alarmes automaticamente."""
    print("\nExecutando coleta de dados...")
    dados = obter_status_nfe()

    if dados:
        salvar_csv(dados)
        print("Coleta concluída.")
    else:
        print("Nenhum dado encontrado!")

# Agendar a execução a cada 10 minutos
schedule.every(10).minutes.do(executar_coleta)

print("Monitoramento iniciado. O programa rodará a cada 10 minutos.")
while True:
    schedule.run_pending()
    time.sleep(1)


Monitoramento iniciado. O programa rodará a cada 10 minutos.

Executando coleta de dados...
Dados salvos em status_nfe.csv
Coleta concluída.


KeyboardInterrupt: 

In [5]:
from google.colab import files
files.download("status_nfe.csv")
files.download("alarmes_nfe.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>